In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2023
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:  Franck P. Vidal (Bangor University, UK)

# <img src="img/gvxr_logo.png" alt="gVXR" width="150"/> a data reader for CIL
 
This demo shows the result of the integration between gVirtualXray (gVXR) and the [Core Imaging Library (CIL)](https://ccpi.ac.uk/cil/) that we developed during the *CIL Training and Bring Your Own Data User Hackathon* at Cambridge University. A cone-beam computed-tomography (CBCT) acquisition with Poisson noise is simulated with [gVXR](https://sourceforge.net/projects/gvirtualxray). The data is reconstructed with [CIL](https://github.com/TomographicImaging/CIL) version 23.0.1.
The anthropomorphic phantom used is from the pEdiatRic dosimetRy personalized platfORm (ERROR) (https://error.upatras.gr/). P. Papadimitroulas et al., "A Review on Personalized Pediatric Dosimetry Applications Using Advanced Computational Tools," in IEEE Transactions on Radiation and Plasma Medical Sciences, vol. 3, no. 6, pp. 607-620, Nov. 2019, doi: [10.1109/TRPMS.2018.2876562](https://doi.org/10.1109/TRPMS.2018.2876562)."
It corresponds to a 5-year old boy. It is available at https://gate.uca.fr/download/examples-tools. 

Author: Franck Vidal

Version: 1.2, 8 Nov 2023

# Aims of this session

- Simulate a CBCT scan acquisition using gVXR;
- Add Poisson noise corresponding to a given number of photons per pixel; and
- Reconstruct the CT volume using the [Core Imaging Library (CIL)](https://ccpi.ac.uk/cil/).


In our simulation the source-to-object distance (SOD) is 1000mm, and the source-to-detector distance (SDD) is 1125mm. The beam spectrum is polychromatic. The voltage is 85 kV. The filtration is 0.1 mm of copper and 1 mm of aluminium. The energy response of the detector is considered. It mimics a 600-micron thick CsI scintillator. 15,000 photons per pixels are used. 600 projections of 512x512 pixels are taken.

![Main parameters of the simulation](img/pediatric-setup.png)

# Main steps

1. Download the phantom data. Anthropomorphic data is used. It corresponds to a 5-year old boy. 

2. Extract surface meshes from the voxelied phantom.

3. Simulate an X-ray radiograph of the virtual patient.

![Corresponding radiograph](./output/visualisation.png)

4. Select the number of incident photons per pixel.

5. Add the corresponding amount of Photonic noise.

![X-ray projection with Poisson noise](output/noisy-projection.png)

6. Create the flat-field images with the corresponding amount of Photonic noise.

![Average flat-field image with Poisson noise](./output/average-flat-field.png)

7. Simulate a CT scan.

![Scanning eometry](output/CT-geometry.png)

8. Reconstruct the CT volume using the [Core Imaging Library (CIL)](https://ccpi.ac.uk/cil/).

![Visualisation of the reconstructed 3D volume](./output/plotCT.png)

In [ ]:
%matplotlib inline

The working directory is not necessary the path of the Notebook on my Mac. 
Use either:

```python
import pathlib
root_path = str(pathlib.Path().resolve())
```

or 

```python
root_path = str(globals()['_dh'][0])
```

to locate the path of the notebbok. This is useful to save output files.

In [ ]:
root_path = str(globals()['_dh'][0])

# Create directories

This step is needed to store some files.

In [ ]:
import os

def createDirectory(directory):
    # The directory does not exist
    if not os.path.exists(os.path.abspath(directory)):

        # Create the directory
        os.mkdir(os.path.abspath(directory))

createDirectory(root_path + "/input_data")
createDirectory(root_path + "/input_data/meshes")
createDirectory(root_path + "/output")

# Import packages

See [environment.yml](environment.yml) for a Conda environment file.

In [ ]:
import glob
import zipfile
import urllib

import pandas as pd

from IPython.display import display
from IPython.display import Image

import matplotlib.pyplot as plt # Plotting
import numpy as np

from tifffile import imread, imwrite

from IPython.display import display
from IPython.display import Image

import SimpleITK as sitk

import matplotlib # To plot images

font = {'family' : 'serif',
         'size'   : 10
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

from tqdm.contrib import tzip


from ipywidgets import interact
import ipywidgets as widgets

import base64

from gvxrPython3 import gvxr
from gvxrPython3 import json2gvxr
from gvxrPython3.utils import visualise
from gvxrPython3.utils import plotScreenshot
from gvxrPython3.utils import interactPlotPowerLaw # Plot the X-ray image using a Power law look-up table
from gvxrPython3.utils import saveProjections # Plot the X-ray image in linear, log and power law scales
# gvxr.useLogFile()

from sitk2vtk import *
from reconstruct import *

if has_cil:
    from gvxrPython3.JSON2gVXRDataReader import *
    from cil.utilities.display import show_geometry
    from cil.utilities.jupyter import islicer

## Initialise GVXR using our JSON file

In [ ]:
json_fname = root_path + "/Noise-CBCT.json"

# MS Windows
if os.name == "nt":
    json2gvxr.initGVXR(json_fname, renderer="EGL")
# MacOS
elif str(os.uname()).find("Darwin") >= 0:
    json2gvxr.initGVXR(json_fname, renderer="OPENGL")
# GNU/Linux
else:
    json2gvxr.initGVXR(json_fname, renderer="EGL")

## Create the output directory

In [ ]:
raw_projection_output_dir = os.path.abspath(json2gvxr.getFilePath(json2gvxr.params["Scan"]["OutFolder"]))
print("The raw projections were saved in", raw_projection_output_dir)
createDirectory(raw_projection_output_dir + "/..")
createDirectory(raw_projection_output_dir)

## Load our detector

In [ ]:
json2gvxr.initDetector()

In [ ]:
number_of_rows = json2gvxr.params["Detector"]["NumberOfPixels"][1]
number_of_cols = json2gvxr.params["Detector"]["NumberOfPixels"][0]

## Load our source properties

In [ ]:
json2gvxr.initSourceGeometry()

In [ ]:
spectrum, unit_of_energy, energy_set, bin_sets = json2gvxr.initSpectrum(verbose=0)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(energy_set, bin_sets)
plt.xlabel("Energy [" + unit_of_energy + "]")
plt.ylabel("Photon count")
plt.title("Corresponding spectra")

# Step 1. Download and extract the phantom data from a ZIP file.

In [ ]:
if not os.path.exists("input_data/Pediatric phantom.zip"):
    urllib.request.urlretrieve("https://drive.uca.fr/f/384a08b5f73244cf9ead/?dl=1",
                               root_path + "/input_data/Pediatric phantom.zip")

    with zipfile.ZipFile(root_path + "/input_data/Pediatric phantom.zip","r") as zip_ref:
        zip_ref.extractall(root_path + "/input_data")

# Step 2. Extract surface meshes from the voxelied phantom.

Load the phantom

In [ ]:
phantom = sitk.ReadImage(root_path + "/input_data/Pediatric phantom/Pediatric_model.mhd")

Load the labels

In [ ]:
df = pd.read_csv(root_path + "/input_data/labels.dat")

Process every structure of the phantom

In [ ]:
meshes = []

for threshold, organ in tzip(df["Label"], df["Organs"],
                         desc="Processing anatomy"):

    # Ignore air
    if organ != "Air":

        mesh_fname = root_path + "/input_data/meshes/" + organ + ".stl"
        meshes.append(mesh_fname)

        # Only create the mesh if it does not exist
        if not os.path.exists(mesh_fname):

            # Threshold the phantom
            binary_image = (phantom == threshold)

            # Smooth the binary segmentation
            smoothed_binary_image = sitk.AntiAliasBinary(binary_image)

            # Create a VTK image
            vtkimg = sitk2vtk(smoothed_binary_image, centre=True)

            vtk_mesh = extractSurface(vtkimg, 0)
            writeSTL(vtk_mesh, mesh_fname)

In [ ]:
del phantom

## Load our samples

In [ ]:
json2gvxr.initSamples(verbose=0)

In [ ]:
gvxr.moveToCentre()

ID = "root"
min_x, min_y, min_z, max_x, max_y, max_z = gvxr.getNodeAndChildrenBoundingBox(ID, "mm")
centre_x = (min_x + max_x) / 2.0
centre_y = (min_y + max_y) / 2.0
centre_z = (min_z + max_z) / 2.0

print("Bounding box:", [min_x, min_y, min_z], [max_x, max_y, max_z])
print("Bounding box centre:", [centre_x, centre_y, centre_z])

# Step 3. Simulate an X-ray radiograph of the virtual patient.

We create an X-ray image `projection_in_MeV`.
By default the image is expressed in MeV.
We convert it to keV for display as follows: `projection_in_keV = projection_in_MeV / gvxr.getUnitOfEnergy("keV")`.

In [ ]:
projection_in_MeV = np.array(gvxr.computeXRayImage(), dtype=np.single)
projection_in_keV = projection_in_MeV / gvxr.getUnitOfEnergy("keV")

In [ ]:
gvxr.setWindowSize(500, 500) # Fix for MacOS
gvxr.displayScene()
plotScreenshot()

In [ ]:
fname = root_path + "/output/visualisation.png"

if not os.path.exists(fname):

    plot = visualise(use_log=True)
    plot.grid_visible = False

    plot.display()
else:
    display(Image(fname, width=800))

In [ ]:
if not os.path.exists(fname):
    if plot is not None:

        plot.fetch_screenshot()

        data = base64.b64decode(plot.screenshot)
        with open(fname,'wb') as fp:
            fp.write(data)

# Step 4. Select the number of incident photons per pixel

<!--
1. Load the raw projection in the RAM
2. Convert the image in keV or MeV into number of photons
3. Add the Poisson noise
4. Convert the image in number of photons into keV or MeV
5. Apply the flat-field correction -->

In [ ]:
fig_plot = None
def chooseNumberOfPhotonsPerPixel(xray_image: np.array, number_of_photons_per_pixel:int=15000, figsize=(10, 5)):

    """
    Use Matplotlib and a Jupyter widget to display the X-ray image with Poisson noise.
    The number of photons per pixel can be change interactively.

    @param xray_image: The image to display
    @number_of_photons_per_pixel: the number of photons per pixel (default: 15000)
    @gamma figsize: the size of the figure (default: (10, 5))
    """

    global target_number_of_photons_per_pixel, fig_plot
    target_number_of_photons_per_pixel = number_of_photons_per_pixel

    noisy_image = getNoisyImage(xray_image, number_of_photons_per_pixel)

    fig_plot, axs = plt.subplots(1, 2, figsize=figsize)
    ax_img = axs[0]
    
    ax_img.set_xticks([])
    ax_img.set_yticks([])

    ax_plt = axs[1]
    img = ax_img.imshow(noisy_image, cmap="gray")
    ax_img.plot([0, noisy_image.shape[1]], [0, noisy_image.shape[0]])
    profile, = ax_plt.plot(np.diag(noisy_image))
    
    # cbar = fig_plot.colorbar(img, orientation='vertical')
    title_str = "Photons per pixels: " + str(number_of_photons_per_pixel)
    ax_img.set_title(title_str)
    ax_plt.set_title("Diagonal profile")
    ax_plt.set_xlabel("Pixel position")
    ax_plt.set_ylabel("Integrated energy in MeV")
    plt.tight_layout()
    plt.margins(0,0)

    plt.close()

    ## Callback function: plot y=Acos(x+phi)
    def update_plot(number_of_photons_per_pixel):
        global target_number_of_photons_per_pixel
        target_number_of_photons_per_pixel = number_of_photons_per_pixel
        noisy_image = getNoisyImage(xray_image, number_of_photons_per_pixel)
        img = ax_img.imshow(noisy_image, cmap="gray")
        title_str = "Photons per pixels: " + str(number_of_photons_per_pixel)
        ax_img.set_title(title_str)
        # fig_plot.colorbar(img, cax=cbar.ax, orientation='vertical')
        
        profile_data = np.diag(noisy_image)
        profile.set_ydata(profile_data)
        ax_plt.set_ylim((0, profile_data.max()))

        display(fig_plot)

    interact(update_plot,
             number_of_photons_per_pixel=widgets.IntSlider(value=number_of_photons_per_pixel, min=10, max=50000, step=10, description="Photons/pixels"))

# Step 5. Add the corresponding amount of Photonic noise 

In [ ]:
gvxr.enablePoissonNoise()

def getNoisyImage(x_ray_image_energy, target_number_of_photons_per_pixel):
    gvxr.setNumberOfPhotons(target_number_of_photons_per_pixel)
    return np.array(gvxr.computeXRayImage(), dtype=np.single)

You may use the slider to specify the number of photons emitted towards each pixel of the detector. It controls the amount of noise in the image. The simulated image will be very noisy if a small number is used.

In [ ]:
chooseNumberOfPhotonsPerPixel(projection_in_MeV, number_of_photons_per_pixel=15000, figsize=(10, 5))

In [ ]:
fig_plot.savefig(root_path + "/output/noisy-projection.png", dpi=72)

In [ ]:
print("Photons per pixels:", target_number_of_photons_per_pixel)

# Step 6. Create the flat-field images with the corresponding amount of Photonic noise.

Create the flat field image. You may use the slider to specify the number of white images used in the flat-field correction.

In [ ]:
white_slider = widgets.IntSlider(value=25, min=1, max=100, step=1, description='Number of flat images:')
white_slider

In [ ]:
print("Number of flat images:", white_slider.value)
json2gvxr.params["Scan"]["NumberOfWhiteImages"] = white_slider.value

In [ ]:
fname = root_path + "/output/flat.tif"

flats = []

for i in range(white_slider.value):
    flats.append(gvxr.getWhiteImage())

flat_field = np.average(flats, axis=0)

imwrite(fname, flat_field.astype(np.single), compression='zlib')

In [ ]:
total_energy_MeV = gvxr.getTotalEnergyWithDetectorResponse()


fig = plt.figure(figsize = (10, 10))
plt.title("Average flat image\n(" + str(white_slider.value) + " images, " + str(target_number_of_photons_per_pixel) + " photons per pixel)")
img = plt.imshow(flat_field, cmap='gray')
plt.tight_layout()
plt.savefig(root_path + "/output/average-flat-field.png", dpi=72)

# Step 7. Simulate a CT scan

In [ ]:
angles = json2gvxr.initScan()

In [ ]:
number_of_angles = json2gvxr.params["Scan"]["NumberOfProjections"]
angles = json2gvxr.doCTScan()

In [ ]:
if json2gvxr.white_image is not None:
    white_image = json2gvxr.white_image
    fname = root_path + "/output/flat.tif"
    imwrite(fname, flat_field.astype(np.single), compression='zlib')

In [ ]:
print("First angle:", angles[0])
print("Last angle:", angles[-1])
print("Number of angles:", number_of_angles)

# Step 8. Reconstruct the CT volume using the [Core Imaging Library (CIL)](https://ccpi.ac.uk/cil/)

In [ ]:
reader = JSON2gVXRDataReader(file_name=json_fname)
data = reader.read()
reconstruction = reconstruct(data, "CONEBEAM", False, verbose=1)

In [ ]:
if has_cil:
    fig = show_geometry(data.geometry)
    fig.save(root_path  + "/Noise-CBCT/CT-geometry.png", dpi=72)

In [ ]:
islicer(reconstruction, direction='vertical')

In [ ]:
islicer(reconstruction, direction='horizontal_x')

In [ ]:
reconstruction_as_array = reconstruction.as_array()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15, 7))
plt.suptitle("CBCT reconstruction with noise and polychromatism")
ax1.imshow(reconstruction_as_array[ int(reconstruction_as_array.shape[0] * 0.5), :, :], cmap='gray')
ax2.imshow(reconstruction_as_array[ :, int(reconstruction_as_array.shape[1] * 0.25), :], cmap='gray')
ax3.imshow(reconstruction_as_array[ :, :, int(reconstruction_as_array.shape[2] * 0.5)], cmap='gray')
plt.subplots_adjust(top=1.25)
plt.savefig(root_path  + "/output/plotCT.png", dpi=72)
plt.show()

## Save the volume in a file using [SimpleITK](https://simpleitk.org/)

In [ ]:
fname = root_path  + "/output/CT_in_mu.mha"

In [ ]:
detector_size = np.array(gvxr.getDetectorSize("mm"))
number_of_pixels = np.array(gvxr.getDetectorNumberOfPixels())
spacing = detector_size / number_of_pixels

print("CT volume saved in", fname)
sitk_image = sitk.GetImageFromArray(reconstruction_as_array)
sitk_image.SetSpacing([spacing[0], spacing[0], spacing[1]])
sitk.WriteImage(sitk_image, fname, useCompression=True)

# Cleaning up

Once we have finished it is good practice to clean up the OpenGL contexts and windows with the following command.

In [ ]:
gvxr.terminate()